### import libraries

In [ ]:
import numpy as np
import scipy.sparse as sp
from scipy.io import savemat # For matlab!
import os
from Bio import PDB

### set parameters

In [ ]:
'''
@params
save_dir: where you want the output to be saved
pdb_f: path to pdb input
chain: chain id to use
'''
save_dir = f"./arsa_graphs"
pdb_f = f"{save_dir}/AF_arsa.pdb"
chain = "A"

### read pdb, make graph & matlab input

In [ ]:
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

edge_dist_threshold = 4.5 # 4.5 Å for any atom pair
three_letter_to_one = {
    "Ala": "A", "Arg": "R", "Asn": "N", "Asp": "D",
    "Cys": "C", "Gln": "Q", "Glu": "E", "Gly": "G",
    "His": "H", "Ile": "I", "Leu": "L", "Lys": "K",
    "Met": "M", "Phe": "F", "Pro": "P", "Ser": "S",
    "Thr": "T", "Trp": "W", "Tyr": "Y", "Val": "V",
    "Sec": "U", "Pyl": "O", "Asx": "B", "Glx": "Z",
    "Xaa": "X", "Ter": "*"
}
pdb = pdb_f.split('/')[-1].split('.')[0]
parser = PDB.PDBParser(QUIET=True)

model = parser.get_structure(pdb, pdb_f)[0][chain]

'''
get residues and coords
'''
aa_labels = []
coords = []
num_residues = 0
for residue in model:
    # only get standard amino acids, not ligand or water
    if PDB.is_aa(residue):
        num_residues += 1
        if residue.get_resname().capitalize() in three_letter_to_one:
            aa_labels.append(three_letter_to_one[residue.get_resname().capitalize()])
        else:
            aa_labels.append('X')
        # print(aa_labels[-1])
        atom_coords = {}
        for atom in residue:
            atom_coords[atom.get_name()] = residue[atom.get_name()].coord
        coords.append(atom_coords)

'''
construct edge matrix, no self-loops
'''
edge_mat = np.zeros((num_residues,num_residues)) # dont include diagonal for pedja's profile kernel input
for i in range(num_residues):
    for j in range(i+1,num_residues):
         # just don't include edges for nodes that don't have any atoms
        if i < len(coords) and j < len(coords) and coords[i] is not None and coords[j] is not None:
            found = False
            for atom_i in coords[i]:
                for atom_j in coords[j]:
                    if not found:
                        dist = np.linalg.norm(coords[i][atom_i] - coords[j][atom_j])
                        if dist <= edge_dist_threshold: 
                            # print(f'{i},{j}: atoms {atom_i} and {atom_j} <= 4.5Å')
                            assert edge_mat[i,j] == edge_mat[j,i] and edge_mat[i,j] == 0
                            edge_mat[i,j],edge_mat[j,i] = 1,1 # undirected edges
                            found = True

'''
save to matlab input file
assume matlab automatically converts zero-based indexing to one-based
'''
savemat(f'{save_dir}/{pdb}_{chain}.mat', {'G': sp.csr_matrix(edge_mat), 'L': aa_labels}) # aa labels is the same as pdb seq as a list

In [ ]:
len(aa_labels),len(coords)

### make jose's graphlet kernel input

In [ ]:
from scipy.io import loadmat

f = f"{save_dir}/{pdb}_{chain}.mat"
key = f.split('/')[-1].split('.')[0]
mat_data = loadmat(f'{save_dir}/{key}.mat')

dense_mat = mat_data['G'].toarray()
pdb_seq = mat_data['L']
if len(pdb_seq) == 1:
    pdb_seq = pdb_seq[0]
else:
    pdb_seq = ''.join(pdb_seq)

with open(f"{save_dir}/{key}.graph",'w') as f_w:
    for i in range(len(dense_mat)):
        nonzero = dense_mat[i].nonzero()
        assert len(nonzero) == 1
        close_indices = '\t'.join(map(str, nonzero[0]))
        f_w.write(f"{i} {close_indices}\n")
    print(f"wrote {save_dir}/{key}.graph")

with open(f"{save_dir}/{key}.labels",'w') as f_w:
    f_w.write(pdb_seq)
    print(f"wrote {save_dir}/{key}.labels")

### write pos & neg idx files

In [ ]:
'''
fix as needed for your data
if your data is not labeled pos/neg, just set each point to whichever classification
jose's graphlet kernel input will only generate features for indices either in the .pos or .neg file,
    so put all indices if you want features for all residues
'''
with open(f'{save_dir}/{key}.pos','w') as f:
    for res_idx in range(len(aa_labels)):
        f.write(f"{res_idx}\n") # normal indexing for jose's input
    print(f'wrote {save_dir}/{key}.pos')
with open(f'{save_dir}/{key}.neg','w') as f:
    print(f'wrote {save_dir}/{key}.neg')